# **Real Estate price prediction (Sberbank)**

In [ ]:
import scipy as sc
import numpy as np
import pandas as pd
import sklearn.ensemble
import random

## Macro data preparation

To fill means instead of nulls

In [ ]:
macro = pd.read_csv('../input/macro.csv', sep = ',', header = 0)
macro.head()

In [ ]:
macro.rename(columns={'rent_price_4+room_bus':'rent_price_4room_bus'}, inplace=True)

In [ ]:
macro.child_on_acc_pre_school.value_counts()

In [ ]:
k=0
msum=0.
for i in range(macro.shape[0]):   
    chld= str(macro.child_on_acc_pre_school[[i]].values)
    if pd.isnull(macro.child_on_acc_pre_school[[i]].values)==False :
        chld=chld[2:-2]
    else:
        chld=chld[1:-1]
    
    if chld!='#!' or pd.isnull(macro.child_on_acc_pre_school[[i]].values)==True :
        
        chld=str.replace(chld, "," , ".")
        macro.set_value( i,'child_on_acc_pre_school',float(chld) )
        if pd.isnull(macro.child_on_acc_pre_school[[i]].values)==False:
            k+=1
            
            msum+=float(chld)

for i in range(macro.shape[0]):
    chld= str(macro.child_on_acc_pre_school[[i]].values)
    chld=chld[2:-2]
    if chld=='#!':
        val=  str((ord('#'))+ord('!')*0.01)
        chld=str.replace(chld, "#!" , val)
        macro.set_value( i,'child_on_acc_pre_school',float(chld) )
        k+=1
        msum+=float(chld)

val=str(msum/k)


for i in range(macro.shape[0]):
    if pd.isnull(macro.child_on_acc_pre_school[[i]].values)==True:
        chld=val
        macro.set_value( i,'child_on_acc_pre_school',float(chld) )

In [ ]:
macro.child_on_acc_pre_school.value_counts() #got full data

In [ ]:
macro.modern_education_share.value_counts()

In [ ]:
k=0
msum=0.
for i in range(macro.shape[0]):   
    chld= str(macro.modern_education_share[[i]].values)
    if pd.isnull(macro.modern_education_share[[i]].values)==False :
        chld=chld[2:-2]
        chld=str.replace(chld, "," , ".")
        macro.set_value( i,'modern_education_share',float(chld) )
        k+=1
        msum+=float(chld)
val=str(msum/k)
for i in range(macro.shape[0]):
    if pd.isnull(macro.modern_education_share[[i]].values)==True:
        chld=val
        macro.set_value( i,'modern_education_share',float(chld) )

In [ ]:
macro.modern_education_share.value_counts() #got full data

In [ ]:
macro.old_education_build_share.value_counts()

In [ ]:
for i in range(macro.shape[0]):
    if pd.isnull(macro['provision_retail_space_modern_sqm'][[i]].values) != False:
                 macro.set_value( i,'provision_retail_space_modern_sqm',(690.+271)/2 )

#macro['provision_retail_space_modern_sqm'] = \
#map(lambda x: x if pd.isnull(x)== False else (690.+271)/2, macro['provision_retail_space_modern_sqm'].values)
macro.provision_retail_space_modern_sqm.value_counts()

In [ ]:
for i in range(1,macro.shape[1]): #start i=1, macro[[0]] is date and this column is full
    d = str(macro[[i]].columns.values)
    d=d[2:-2]
    mc= (lambda x: x if pd.isnull(x)== False else 0, macro[d].values)
    
    ms=0.
    k=0
    for j in range( len(mc)):
        if pd.isnull(mc[1][j])==False and type(mc[1][j])!=map:
            
            ms=ms+mc[1][j]
            k+=1
        else:
            k=len(mc)
    mm=ms/k
    macro[d].fillna(mm,  inplace=True)

In [ ]:
shift =30
for i in range(macro.shape[0]):
    if (i+shift) <macro.shape[0]:
        macro.set_value( i,'timestamp',macro.timestamp[i+shift])
        
    else:
        macro.set_value( i,'timestamp',macro.timestamp[macro.shape[0]-i])

## Train data preparation

filling means/medians or zeros or new values instead of nulls (depends of data application)


change categorial marks to numeric

In [ ]:
train = pd.read_csv('../input/train.csv', sep = ',', header = 0)
train.head()

In [ ]:
train.build_year.value_counts() #correction of build year mistakes

**Assumptions:**

4965 --> maybe 1965

2018--> maybe the buiding on the construction

1691--> 1991

215 --> 2015

71, 20 --> 1971, 1920

20052009 --->2009

0, 1, 3 ---> 2000, 2001, 2003

In [ ]:
for i in range(train.shape[0]):   
    if train.build_year[[i]].values==4965 :
        train.set_value( i,'build_year',int(1965) )
    elif train.build_year[[i]].values==1691 :
        train.set_value( i,'build_year',int(1991) )
    elif train.build_year[[i]].values==215 :
        train.set_value( i,'build_year',int(2015) )
    elif train.build_year[[i]].values==71 :
        train.set_value( i,'build_year',int(1971) )
    elif train.build_year[[i]].values==20 :
        train.set_value( i,'build_year',int(1920) )
    elif train.build_year[[i]].values==0 :
        train.set_value( i,'build_year', int(2000) )
    elif train.build_year[[i]].values==1 :
        train.set_value( i,'build_year',int(2001) )
    elif train.build_year[[i]].values==3 :
        train.set_value( i,'build_year',int(2003) )
    elif train.build_year[[i]].values==20052009 :
        train.set_value( i,'build_year',int(2009) )   
    elif pd.isnull(train.build_year[[i]].values)==True :
        train.set_value( i,'build_year',int(-1) )

set unreal marks for NULL accomodation parameters

In [ ]:
def set_unreal(collumn_list, data_array):
    for c in collumn_list:
        data_array[c].fillna(-1, inplace=True)
        #= (lambda x: -1. if pd.isnull(x) == True else x ,data_array[c] ) #data_array[c]
        
        #for j in range (data_array.shape[0]):
        #    if pd.isnull(data_array[c][[j]].values)==True :
        #        data_array.set_value( j, c ,-1. )

In [ ]:
accom_param =['full_sq', 'life_sq','floor', 'max_floor', 'material', 'num_room', 'kitch_sq','state', \
              'preschool_education_centers_raion', 'ID_railroad_station_walk',\
             'school_education_centers_raion', 'school_education_centers_top_20_raion', 'healthcare_centers_raion', \
             'university_top_20_raion', 'sport_objects_raion', 'additional_education_raion', \
              'culture_objects_top_25', \
             'shopping_centers_raion', 'office_raion'] 

In [ ]:
set_unreal(accom_param, train)

no/yes identificaton

In [ ]:
yesno=['culture_objects_top_25','thermal_power_plant_raion', 'incineration_raion', 'oil_chemistry_raion', \
      'radiation_raion', 'railroad_terminal_raion', 'big_market_raion', 'nuclear_reactor_raion', \
      'detention_facility_raion', 'water_1line', 'big_road1_1line', 'railroad_1line']

set means for build count parameters

In [ ]:
def set_means_with_proportion (ammount, collumn_list, data_array):
    bcM=[]
    meanRb = (lambda x: x if pd.isnull(x)== False else 0., data_array[ammount])[1]
    meanRb = sum(meanRb)
    k= len(data_array[ammount].values.tolist())- data_array[ammount].values.tolist().count(None)
    meanRb =meanRb/ k
    # mean of estimated cells
    for j in range(len(collumn_list)):   
        meanb = (lambda x, y: x/y if pd.isnull(x)== False and y!=0. else 0., data_array[collumn_list[j]] , data_array[ammount])[1]
        meanb = sum (meanb)
        bcM.append(meanb/k) #mean of depended cells
    data_array[ammount] = (lambda x: int(meanRb) if pd.isnull(x)== True else x, data_array[ammount])[1]
    #set mean of ammount cell
    for j in range(len(collumn_list)):
        data_array[collumn_list[j]] = (lambda x: int(meanRb *bcM[j]) if pd.isnull(x)== True else x, data_array[collumn_list[j]])[1]

        

In [ ]:
bc=['build_count_block', 'build_count_wood', 'build_count_frame', 'build_count_brick', \
    'build_count_monolith',  'build_count_panel', 'build_count_foam', 'build_count_slag', \
    'build_count_mix']



    

In [ ]:
set_means_with_proportion('raion_build_count_with_material_info', bc, train)

In [ ]:
bi = ['build_count_before_1920', 'build_count_1921-1945', \
      'build_count_1946-1970', 'build_count_1971-1995', 'build_count_after_1995']

In [ ]:
set_means_with_proportion('raion_build_count_with_builddate_info', bi, train)

In [ ]:
means_only =['cafe_sum_500_min_price_avg','cafe_sum_500_max_price_avg','cafe_avg_price_500',\
            'cafe_sum_1000_min_price_avg','cafe_sum_1000_max_price_avg','cafe_avg_price_1000',\
            'cafe_sum_1500_min_price_avg','cafe_sum_1500_max_price_avg','cafe_avg_price_1500',\
            'cafe_sum_2000_min_price_avg','cafe_sum_2000_max_price_avg','cafe_avg_price_2000',\
            'cafe_sum_3000_min_price_avg','cafe_sum_3000_max_price_avg','cafe_avg_price_3000',\
            'prom_part_5000', \
            'cafe_sum_5000_min_price_avg','cafe_sum_5000_max_price_avg','cafe_avg_price_5000']

In [ ]:
def set_means (collumn_list, data_array):
    for j in range(len(collumn_list)):
        meanC = (lambda x: x if pd.isnull(x)== False else 0., data_array[collumn_list[j]])[1]
        meanC = sum(meanC)
        k= len(data_array[collumn_list[j]].values.tolist())- data_array[collumn_list[j]].values.tolist().count(None)
        meanC =meanC/ k
        data_array[collumn_list[j]] = (lambda x: float(meanC) if pd.isnull(x)== True else x, data_array[collumn_list[j]])[1]

In [ ]:
set_means(means_only, train)

In [ ]:
ps=['preschool_quota', 'school_quota', 'hospital_beds_raion', 'metro_min_walk','metro_km_walk',\
   'railroad_station_walk_km', 'railroad_station_walk_min']

In [ ]:
set_means(ps, train)

In [ ]:
def rename_digital_col_names(df):
    df.rename(columns={'0_6_all': 'zero_6_all', '0_6_male':'zero_6_male', '0_6_female': 'zero_6_female',
                          '7_14_all': 'seven_14_all','7_14_male': 'seven_14_male', '7_14_female': 'seven_14_female',\
                          '0_17_all':'zero_17_all','0_17_male':'zero_17_male',\
                          '0_17_female':'zero_17_female',\
                          '16_29_all':'sixteen_29_all','16_29_male':'sixteen_29_male',\
                          '16_29_female':'sixteen_29_female','0_13_all':'zero_13_all',\
                          '0_13_male':'zero_13_male','0_13_female':'zero_13_female'}, inplace=True)
    df.rename(columns={'build_count_1921-1945':'build_count_1921_1945','build_count_1946-1970': 'build_count_1946_1970',\
                          'build_count_1971-1995':'build_count_1971_1995'}, inplace=True)

In [ ]:
rename_digital_col_names(train)

In [ ]:
#add an adition mark for "product type" as "Unknown"
k = random.sample(range(train.shape[0]), 1000)
k=sorted(k)
for i in k:
    train.set_value( i,'product_type',u'Unknown' )

In [ ]:
cafe_count500 =['cafe_count_500_na_price','cafe_count_500_price_500','cafe_count_500_price_1000',\
               'cafe_count_500_price_1500',\
               'cafe_count_500_price_2500','cafe_count_500_price_4000','cafe_count_500_price_high']
cafe_count1000=['cafe_count_1000_na_price','cafe_count_1000_price_500','cafe_count_1000_price_1000',\
                'cafe_count_1000_price_1500','cafe_count_1000_price_2500','cafe_count_1000_price_4000',\
                'cafe_count_1000_price_high']
cafe_count1500=['cafe_count_1500_na_price','cafe_count_1500_price_500','cafe_count_1500_price_1000',\
                 'cafe_count_1500_price_1500','cafe_count_1500_price_2500','cafe_count_1500_price_4000',\
                 'cafe_count_1500_price_high']

cafe_count2000=['cafe_count_2000_na_price','cafe_count_2000_price_500','cafe_count_2000_price_1000',\
                'cafe_count_2000_price_1500','cafe_count_2000_price_2500','cafe_count_2000_price_4000',\
                'cafe_count_2000_price_high']
cafe_count3000=['cafe_count_3000_na_price','cafe_count_3000_price_500','cafe_count_3000_price_1000',\
                'cafe_count_3000_price_1500','cafe_count_3000_price_2500','cafe_count_3000_price_4000',\
                'cafe_count_3000_price_high']
cafe_count5000=['cafe_count_5000_na_price','cafe_count_5000_price_500','cafe_count_5000_price_1000',\
                'cafe_count_5000_price_1500','cafe_count_5000_price_2500','cafe_count_5000_price_4000',\
                'cafe_count_5000_price_high']

CC_id =['cafe_count_500', 'cafe_count_1000', 'cafe_count_1500', 'cafe_count_2000', \
       'cafe_count_3000', 'cafe_count_5000']
CC= [cafe_count500, cafe_count1000, cafe_count1500, cafe_count2000, cafe_count3000, cafe_count5000]

In [ ]:
def to_float(ammount_column, depended_columns, data_array):
    depended_columns.insert(0, ammount_column)
    for c in depended_columns:
        data_array[c]= (lambda x: float(x) , data_array[c])[1]
        #for j in range (data_array.shape[0]):
        #    data_array.set_value( j, c ,float(data_array[c][[j]].values) )

In [ ]:
for i in range(len(CC_id)):
    to_float (CC_id[i], CC[i], train)

In [ ]:
def dependeds_to_means (ammount_column, depended_columns, data_array):
    for c in depended_columns:
        data_array[c]=(lambda x, y: x/y if y !=0. else y, data_array[c], data_array[ammount_column])[1]

In [ ]:
dependeds_to_means('raion_build_count_with_material_info', bc, train)

In [ ]:
bi_corr = ['build_count_before_1920', 'build_count_1921_1945', \
      'build_count_1946_1970', 'build_count_1971_1995', 'build_count_after_1995']

In [ ]:
dependeds_to_means('raion_build_count_with_builddate_info', bi_corr, train)

In [ ]:
for i in range(len(CC_id)):
    dependeds_to_means (CC_id[i], CC[i], train)

In [ ]:
train.head()

In [ ]:
full_train = pd.merge(train, macro, on='timestamp')
timestamp_price= full_train.timestamp
full_train = full_train.drop(['timestamp'], 1)

In [ ]:
full_train.head()

In [ ]:
categorical_columns = [c for c in full_train.columns if full_train[c].dtype.name == 'object']

additional_categorical_columns =['build_year', 'ID_metro', 'ID_railroad_station_walk', \
                                 'ID_railroad_station_avto', 'ID_big_road1', 'ID_big_road2','ID_railroad_terminal',\
                                'ID_bus_terminal']
categorical_columns = categorical_columns + additional_categorical_columns


col_al=full_train.columns.values.tolist()
min_col=bi_corr+ bc+ accom_param+ means_only+ps+categorical_columns+yesno
min_col.append('build_year')
#min_col.insert(0,'timestamp')
min_col.insert(0,'id')


NN2=[]
for i in min_col:
    if not i in NN2:
        NN2.append(i)

In [ ]:
for c in NN2:
    
    col_al.remove(c)
set_means(col_al, full_train)

In [ ]:
d_price=full_train.id #STORE ID-s

In [ ]:
X = full_train.drop(['id', 'price_doc'], 1)
y = full_train.price_doc

In [ ]:
y= (lambda x: float(x) , y)[1]

In [ ]:
numerical_columns   = ['full_sq', 'life_sq', 'floor', 'max_floor', 'material', 'num_room', 'kitch_sq', 'state', 'area_m', 'raion_popul', 'green_zone_part', 'indust_part', 'children_preschool', 'preschool_quota', 'preschool_education_centers_raion', 'children_school', 'school_quota', 'school_education_centers_raion', 'school_education_centers_top_20_raion', 'hospital_beds_raion', 'healthcare_centers_raion', 'university_top_20_raion', 'sport_objects_raion', 'additional_education_raion', 'culture_objects_top_25_raion', 'shopping_centers_raion', 'office_raion', 'full_all', 'male_f', 'female_f', 'young_all', 'young_male', 'young_female', 'work_all', 'work_male', 'work_female', 'ekder_all', 'ekder_male', 'ekder_female', 'zero_6_all', 'zero_6_male', 'zero_6_female', 'seven_14_all', 'seven_14_male', 'seven_14_female', 'zero_17_all', 'zero_17_male', 'zero_17_female', 'sixteen_29_all', 'sixteen_29_male', 'sixteen_29_female', 'zero_13_all', 'zero_13_male', 'zero_13_female', 'raion_build_count_with_material_info', 'build_count_block', 'build_count_wood', 'build_count_frame', 'build_count_brick', 'build_count_monolith', 'build_count_panel', 'build_count_foam', 'build_count_slag', 'build_count_mix', 'raion_build_count_with_builddate_info', 'build_count_before_1920', 'build_count_1921_1945', 'build_count_1946_1970', 'build_count_1971_1995', 'build_count_after_1995', 'metro_min_avto', 'metro_km_avto', 'metro_min_walk', 'metro_km_walk', 'kindergarten_km', 'school_km', 'park_km', 'green_zone_km', 'industrial_km', 'water_treatment_km', 'cemetery_km', 'incineration_km', 'railroad_station_walk_km', 'railroad_station_walk_min', 'railroad_station_avto_km', 'railroad_station_avto_min', 'public_transport_station_km', 'public_transport_station_min_walk', 'water_km','mkad_km', 'ttk_km', 'sadovoe_km', 'bulvar_ring_km', 'kremlin_km', 'big_road1_km', 'big_road2_km', 'railroad_km', 'zd_vokzaly_avto_km', 'bus_terminal_avto_km', 'oil_chemistry_km', 'nuclear_reactor_km', 'radiation_km', 'power_transmission_line_km', 'thermal_power_plant_km', 'ts_km', 'big_market_km', 'market_shop_km', 'fitness_km', 'swim_pool_km', 'ice_rink_km', 'stadium_km', 'basketball_km', 'hospice_morgue_km', 'detention_facility_km', 'public_healthcare_km', 'university_km', 'workplaces_km', 'shopping_centers_km', 'office_km', 'additional_education_km', 'preschool_km', 'big_church_km', 'church_synagogue_km', 'mosque_km', 'theater_km', 'museum_km', 'exhibition_km', 'catering_km', 'green_part_500', 'prom_part_500', 'office_count_500', 'office_sqm_500', 'trc_count_500', 'trc_sqm_500', 'cafe_count_500', 'cafe_sum_500_min_price_avg', 'cafe_sum_500_max_price_avg', 'cafe_avg_price_500', 'cafe_count_500_na_price', 'cafe_count_500_price_500', 'cafe_count_500_price_1000', 'cafe_count_500_price_1500', 'cafe_count_500_price_2500', 'cafe_count_500_price_4000', 'cafe_count_500_price_high', 'big_church_count_500', 'church_count_500', 'mosque_count_500', 'leisure_count_500', 'sport_count_500', 'market_count_500', 'green_part_1000', 'prom_part_1000', 'office_count_1000', 'office_sqm_1000', 'trc_count_1000', 'trc_sqm_1000', 'cafe_count_1000', 'cafe_sum_1000_min_price_avg', 'cafe_sum_1000_max_price_avg', 'cafe_avg_price_1000', 'cafe_count_1000_na_price', 'cafe_count_1000_price_500', 'cafe_count_1000_price_1000', 'cafe_count_1000_price_1500', 'cafe_count_1000_price_2500', 'cafe_count_1000_price_4000', 'cafe_count_1000_price_high', 'big_church_count_1000', 'church_count_1000', 'mosque_count_1000', 'leisure_count_1000', 'sport_count_1000', 'market_count_1000', 'green_part_1500', 'prom_part_1500', 'office_count_1500', 'office_sqm_1500', 'trc_count_1500', 'trc_sqm_1500', 'cafe_count_1500', 'cafe_sum_1500_min_price_avg', 'cafe_sum_1500_max_price_avg', 'cafe_avg_price_1500', 'cafe_count_1500_na_price', 'cafe_count_1500_price_500', 'cafe_count_1500_price_1000', 'cafe_count_1500_price_1500', 'cafe_count_1500_price_2500', 'cafe_count_1500_price_4000', 'cafe_count_1500_price_high', 'big_church_count_1500', 'church_count_1500', 'mosque_count_1500', 'leisure_count_1500', 'sport_count_1500', 'market_count_1500', 'green_part_2000', 'prom_part_2000', 'office_count_2000', 'office_sqm_2000', 'trc_count_2000', 'trc_sqm_2000', 'cafe_count_2000', 'cafe_sum_2000_min_price_avg', 'cafe_sum_2000_max_price_avg', 'cafe_avg_price_2000', 'cafe_count_2000_na_price', 'cafe_count_2000_price_500', 'cafe_count_2000_price_1000', 'cafe_count_2000_price_1500', 'cafe_count_2000_price_2500', 'cafe_count_2000_price_4000', 'cafe_count_2000_price_high', 'big_church_count_2000', 'church_count_2000', 'mosque_count_2000', 'leisure_count_2000', 'sport_count_2000', 'market_count_2000', 'green_part_3000', 'prom_part_3000', 'office_count_3000', 'office_sqm_3000', 'trc_count_3000', 'trc_sqm_3000', 'cafe_count_3000', 'cafe_sum_3000_min_price_avg', 'cafe_sum_3000_max_price_avg', 'cafe_avg_price_3000', 'cafe_count_3000_na_price', 'cafe_count_3000_price_500', 'cafe_count_3000_price_1000', 'cafe_count_3000_price_1500', 'cafe_count_3000_price_2500', 'cafe_count_3000_price_4000', 'cafe_count_3000_price_high', 'big_church_count_3000', 'church_count_3000', 'mosque_count_3000',
 'leisure_count_3000', 'sport_count_3000', 'market_count_3000', 'green_part_5000', 'prom_part_5000', 'office_count_5000', 'office_sqm_5000', 'trc_count_5000', 'trc_sqm_5000', 'cafe_count_5000', 'cafe_sum_5000_min_price_avg', 'cafe_sum_5000_max_price_avg', 'cafe_avg_price_5000', 'cafe_count_5000_na_price', 'cafe_count_5000_price_500', 'cafe_count_5000_price_1000', 'cafe_count_5000_price_1500', 'cafe_count_5000_price_2500', 'cafe_count_5000_price_4000', 'cafe_count_5000_price_high', 'big_church_count_5000', 'church_count_5000', 'mosque_count_5000', 'leisure_count_5000', 'sport_count_5000', 'market_count_5000', 'oil_urals', 'gdp_quart', 'gdp_quart_growth', 'cpi', 'ppi', 'gdp_deflator', 'balance_trade', 'balance_trade_growth', 'usdrub', 'eurrub', 'brent', 'net_capital_export', 'gdp_annual', 'gdp_annual_growth', 'average_provision_of_build_contract', 'average_provision_of_build_contract_moscow', 'rts', 'micex', 'micex_rgbi_tr', 'micex_cbi_tr', 'deposits_value', 'deposits_growth', 'deposits_rate', 'mortgage_value', 'mortgage_growth', 'mortgage_rate', 'grp', 'grp_growth', 'income_per_cap', 'real_dispos_income_per_cap_growth', 'salary', 'salary_growth', 'fixed_basket', 'retail_trade_turnover', 'retail_trade_turnover_per_cap', 'retail_trade_turnover_growth', 'labor_force', 'unemployment', 'employment', 'invest_fixed_capital_per_cap', 'invest_fixed_assets', 'profitable_enterpr_share', 'unprofitable_enterpr_share', 'share_own_revenues', 'overdue_wages_per_cap', 'fin_res_per_cap', 'marriages_per_1000_cap', 'divorce_rate', 'construction_value', 'invest_fixed_assets_phys', 'pop_natural_increase', 'pop_migration', 'pop_total_inc', 'childbirth', 'mortality', 'housing_fund_sqm', 'lodging_sqm_per_cap', 'water_pipes_share', 'baths_share', 'sewerage_share', 'gas_share', 'hot_water_share', 'electric_stove_share', 'heating_share', 'old_house_share', 'average_life_exp', 'infant_mortarity_per_1000_cap', 'perinatal_mort_per_1000_cap', 'incidence_population', 'rent_price_4room_bus', 'rent_price_3room_bus', 'rent_price_2room_bus', 'rent_price_1room_bus', 'rent_price_3room_eco', 'rent_price_2room_eco', 'rent_price_1room_eco', 'load_of_teachers_preschool_per_teacher', 'child_on_acc_pre_school', 'load_of_teachers_school_per_teacher', 'students_state_oneshift', 'modern_education_share', 'old_education_build_share', 'provision_doctors', 'provision_nurse', 'load_on_doctors', 'power_clinics', 'hospital_beds_available_per_cap', 'hospital_bed_occupancy_per_year', 'provision_retail_space_sqm', 'provision_retail_space_modern_sqm', 'turnover_catering_per_cap', 'theaters_viewers_per_1000_cap', 'seats_theather_rfmin_per_100000_cap', 'museum_visitis_per_100_cap', 'bandwidth_sports', 'population_reg_sports_share', 'students_reg_sports_share', 'apartment_build', 'apartment_fund_sqm']

In [ ]:
for c in additional_categorical_columns:
    if c in numerical_columns:
        numerical_columns.remove(c)

In [ ]:
nonbinary_columns = categorical_columns
yesno.append('ID_railroad_station_walk')
for c in yesno:
    nonbinary_columns.remove(c)
yesno.remove('ID_railroad_station_walk')

In [ ]:
data_describe=X.describe(include=[object])

In [ ]:
for c in yesno[0:]:
    
    top = data_describe[c]['top']
    top_items = X[c] == top
    X.loc[top_items, c] = 0
    X.loc[np.logical_not(top_items), c] = 1
    

In [ ]:
data_nonbinary = pd.get_dummies(X[nonbinary_columns])

In [ ]:
#X[numerical_columns].

In [ ]:
data_numerical = X[numerical_columns]
data_numerical.head()

In [ ]:
#print (data_numerical[[1]].values[1])
#print (data_numerical.shape[1])
#print (data_numerical.index.values.tolist() )

In [ ]:
for c in range (data_numerical.shape[1]):
    for i in data_numerical.index.values.tolist():
        if type(data_numerical[[c]].values[i])==str:
            print (c, i, data_numerical[c].values[i])

In [ ]:
data_numerical = (data_numerical - data_numerical.values.mean()) / data_numerical.values.std()

In [ ]:
data_numerical.describe()

In [ ]:
TRAIN_data=pd.concat((data_numerical, X[yesno], data_nonbinary), axis=1) #
TRAIN_data=pd.DataFrame (TRAIN_data, dtype='float')

## Model testind

In [ ]:
print (y)

In [ ]:
id_test= id_price [(id_price.shape[0]-1000):]
X_test=TRAIN_data[(id_price.shape[0]-1000):]
y_test =y[(id_price.shape[0]-1000):]

In [ ]:
X_train=TRAIN_data[:-1000]
y_train =y[:-1000]
id_train= id_price[:-1000]

In [ ]:
X_test=X_test.values
id_test=id_test.values

In [ ]:
X_train=X_train.values
id_train=id_train.values

In [ ]:
print (X_train.shape,  len(y_train))
rf = sklearn.ensemble.RandomForestRegressor (random_state=0)
rf.fit(X_train,  y_train)
y_test2= rf.predict(X_test)

In [ ]:
y_test=np.array(y_test)

### testing RMSLE estimation

In [ ]:
print np.sqrt(sum(((np.log(abs(y_test2)+1.) - np.log(y_test+1.))**2.)/len(y_test)))

# Prediction

### Model education

In [ ]:
rf.fit(TRAIN_data,  y)

### test data preparation

In [ ]:
pred = pd.read_csv('../input/test.csv', sep = ',', header = 0)
pred.head()

In [ ]:
pred.build_year.value_counts() #correction of build year mistakes

Assumptions:


0, 1, 2 ---> 2000, 2001, 2002

In [ ]:
for i in range(pred.shape[0]):   
    if pred.build_year[[i]].values==0 :
        pred.set_value( i,'build_year', int(2000) )
    elif pred.build_year[[i]].values==1 :
        pred.set_value( i,'build_year',int(2001) )
    elif pred.build_year[[i]].values==2 :
        pred.set_value( i,'build_year',int(2002) )   
    elif pd.isnull(pred.build_year[[i]].values)==True :
        pred.set_value( i,'build_year',int(-1) )

set unreal marks for NULL accomodation parameters

In [ ]:
set_unreal(accom_param, pred)

set means for build count parameters

In [ ]:
set_means_with_proportion('raion_build_count_with_material_info', bc, pred)

In [ ]:
set_means_with_proportion('raion_build_count_with_builddate_info', bi, pred)

In [ ]:
set_means(means_only, pred)

In [ ]:
set_means(ps, pred)

In [ ]:
pred['product_type'] = (lambda x: u'Unknown' if pd.isnull(x)==True else x, pred['product_type'])[1]

In [ ]:
rename_digital_col_names(pred) #bi

In [ ]:
for i in range(len(CC_id)):
    to_float (CC_id[i], CC[i], pred)

In [ ]:
dependeds_to_means('raion_build_count_with_material_info', bc, pred)

In [ ]:
dependeds_to_means('raion_build_count_with_builddate_info', bi_corr, pred)

In [ ]:
for i in range(len(CC_id)):
    dependeds_to_means (CC_id[i], CC[i], pred)

In [ ]:
full_pred = pd.merge(pred, macro, on='timestamp')
timestamp_pred= full_pred['timestamp']
full_pred = full_pred.drop(['timestamp'], 1)

In [ ]:
categorical_columns = [c for c in full_pred.columns if full_pred[c].dtype.name == 'object']
#product_type

additional_categorical_columns =['build_year', 'ID_metro', 'ID_railroad_station_walk', \
                                 'ID_railroad_station_avto', 'ID_big_road1', 'ID_big_road2','ID_railroad_terminal',\
                                'ID_bus_terminal']
categorical_columns = categorical_columns + additional_categorical_columns


col_al=full_pred.columns.values.tolist()
min_col=bi_corr+ bc+ accom_param+ means_only+ps+categorical_columns+yesno
min_col.append('build_year')
#min_col.insert(0,'timestamp')
min_col.insert(0,'id')


NN2=[]
for i in min_col:
    if not i in NN2:
        NN2.append(i)

In [ ]:
for c in NN2:
    col_al.remove(c)
set_means(col_al, full_pred)

In [ ]:
id_pred=full_pred['id'] #STORE ID-s

In [ ]:
XP = full_pred.drop(['id'], 1)

In [ ]:
numerical_columns   = [c for c in XP.columns if XP[c].dtype.name != 'object']

In [ ]:
for c in additional_categorical_columns:
    numerical_columns.remove(c)
nonbinary_columns = categorical_columns
for c in yesno:
    nonbinary_columns.remove(c)

In [ ]:

data_describe=XP.describe(include=[object])

In [ ]:
for c in yesno[0:]:
 
    top = data_describe[c]['top']
    top_items = XP[c] == top
    XP.loc[top_items, c] = 0
    XP.loc[np.logical_not(top_items), c] = 1

In [ ]:
data_nonbinary = pd.get_dummies(XP[nonbinary_columns])

In [ ]:
data_numerical = XP[numerical_columns]
data_numerical = (data_numerical - data_numerical.mean()) / data_numerical.std()
data_numerical.describe()

In [ ]:
TEST_data=pd.concat((data_numerical, XP[yesno], data_nonbinary), axis=1) 
TEST_data=pd.DataFrame (TEST_data, dtype='float')

In [ ]:
TEST_data.provision_retail_space_modern_sqm=XP.provision_retail_space_modern_sqm

In [ ]:
sa_PM=pd.DataFrame.from_dict({'sub_area_Poselenie Klenovskoe': np.zeros(TEST_data.shape[0]).tolist()})


TEST_data.join (sa_PM) 
TEST_data['sub_area_Poselenie Klenovskoe']=0.

In [ ]:
y_pred= rf.predict(TEST_data)

In [ ]:
rr={'id' :id_pred,
    'predicted price' : y_pred.tolist()}
RESULT= pd.DataFrame.from_dict(rr)
RESULT.to_csv('Sber RE price prediction/RESULT_data.csv', sep = ',', header=True)

In [ ]:
RESULT